In [1]:
import tensorflow as tf
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Set image dimensions and number of classes
img_width, img_height = 224, 224
num_classes = 2

# Define function to read images from folder and label them
def read_images_from_folder(folder_path, label):
    images = []
    labels = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            img = cv2.imread(os.path.join(folder_path,filename))
            img = cv2.resize(img, (img_width, img_height))
            images.append(img)
            labels.append(label)
    return images, labels

# Read images from folders
normal_images, normal_labels = read_images_from_folder('Normal/', 0)
tb_images, tb_labels = read_images_from_folder('tb/', 1)

# Combine normal and TB images and labels
all_images = normal_images + tb_images
all_labels = normal_labels + tb_labels

# Convert lists to numpy arrays
X = np.array(all_images)
y = np.array(all_labels)

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert labels to one-hot encoding
y_train = to_categorical(y_train, num_classes)
y_val = to_categorical(y_val, num_classes)

# Define the CNN architecture
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
epochs = 10
batch_size = 32

history = model.fit(
    X_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(X_val, y_val))


Epoch 1/10
105/105 [==============================] - 200s 2s/step - loss: 2.4643 - accuracy: 0.8571 - val_loss: 0.2446 - val_accuracy: 0.9226
Epoch 2/10
105/105 [==============================] - 183s 2s/step - loss: 0.2278 - accuracy: 0.9057 - val_loss: 0.1983 - val_accuracy: 0.9298
Epoch 3/10
105/105 [==============================] - 183s 2s/step - loss: 0.1683 - accuracy: 0.9244 - val_loss: 0.1958 - val_accuracy: 0.9393
Epoch 4/10
105/105 [==============================] - 185s 2s/step - loss: 0.1576 - accuracy: 0.9411 - val_loss: 0.1421 - val_accuracy: 0.9476
Epoch 5/10
105/105 [==============================] - 183s 2s/step - loss: 0.1364 - accuracy: 0.9479 - val_loss: 0.2125 - val_accuracy: 0.9310
Epoch 6/10
105/105 [==============================] - 183s 2s/step - loss: 0.1231 - accuracy: 0.9485 - val_loss: 0.1509 - val_accuracy: 0.9440
Epoch 7/10
105/105 [==============================] - 184s 2s/step - loss: 0.1267 - accuracy: 0.9530 - val_loss: 0.1558 - val_accuracy: 0.9536

In [2]:
# Save the model
model.save('my_model.h5')